In [3]:
library("ImpulseDE2")
library('dplyr')
library(ggplot2)

In [11]:
pathSave='/home/khrovatin/timeTrajectoriesNet/data/stages/DE_across_stages/'

In [4]:
genes<-read.table('/home/khrovatin/timeTrajectoriesNet/data/counts/mergedGenes_counts.tsv', header=TRUE,row.names=1, sep="\t")
conditions<-read.table('/home/khrovatin/timeTrajectoriesNet/data/RPKUM/conditions_mergedGenes.tsv', header=TRUE,row.names='Measurment', sep="\t")
#R imported colnames of genes with changes but gene IDs remained ok
rownames(conditions)<-make.names(rownames(conditions))

In [5]:
PHENOTYPES_ORDERED=c('no_agg','disappear', 'stream', 'lag', 'tag', 'tip', 'slug', 'mhat', 'cul', 'tag_spore', 'FB')
PHENOTYPES_X=data.frame(Phenotype=PHENOTYPES_ORDERED,X=c(1:length(PHENOTYPES_ORDERED)))

In [6]:
conditions_annotated=conditions[rowSums(conditions[, PHENOTYPES_ORDERED])>0, ]
X=data.frame(row.names=rownames(genes))
Y=data.frame()
for (row_idx in 1:dim(conditions_annotated)[1]){
  y=conditions_annotated[row_idx,]
  x=genes[,rownames(y)]
  repeated_sample=0
  for (phenotype in PHENOTYPES_ORDERED){
    if (y[phenotype]==1){
      sample_name=paste(rownames(y),repeated_sample,sep='_')
      X[,sample_name]=x
      Y[sample_name,c('Sample','Condition','Batch','Time')]=c(sample_name,'case',y['Replicate'],PHENOTYPES_X[PHENOTYPES_X['Phenotype']==phenotype,'X'])
      repeated_sample=repeated_sample+1
    }
  }
}

In [9]:
objectImpulseDE2 <- runImpulseDE2(matCountData = as.matrix(X), dfAnnotation = Y,boolCaseCtrl = FALSE,vecConfounders = c("Batch"),scaNProc = 20)

ImpulseDE2 for count data, v1.10.0

# Process input



Processing Details:
ImpulseDE2 runs in case-only mode.
Found time points: 1,4,5,6,7,8,9,11,3,2,10
Case: Found the samples at time point 1: FDpool01_TCGCAGG_Hr00_mappedGene_expressions_All_milestone_mRNA_gff_0,FDpool02_TCGCAGG_Hr00_mappedGene_expressions_All_milestone_mRNA_gff_0,Ax4_r1_00h_9_S86_L002_R1_001_mappedGene_expressions_All_milestone_mRNA_gff_0,Ax4_r1_04h_10_S87_L002_R1_001_mappedGene_expressions_All_milestone_mRNA_gff_0,Ax4_r2_00h_9_S58_L003_R1_001_mappedGene_expressions_All_milestone_mRNA_gff_0,Ax4_r2_04h_10_S59_L003_R1_001_mappedGene_expressions_All_milestone_mRNA_gff_0,Pool26_AX4_bio2_hr00_1_TCGCAGG_mappedGene_expressions_All_milestone_mRNA_gff_0,Pool26_AX4_bio2_hr06_7_CTCGATG_mappedGene_expressions_All_milestone_mRNA_gff_0,pool27_AX4_bio1_hr00_1_TCGCAGG_mappedGene_expressions_All_milestone_mRNA_gff_0,pool27_AX4_bio1_hr06_7_CTCGATG_mappedGene_expressions_All_milestone_mRNA_gff_0,Pool28_AX4_bio2_hr00_1_13_ACTATCA_mappedGene_expressions_All_milestone_mRNA_gff_0,pool30_MybBGF

# Run DESeq2: Using dispersion factorscomputed by DESeq2.

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates



[1] "Corrected 69 DESEq2 dispersion estimates which to avoid variance overestimation and loss of discriminatory power for model selection."


Consumed time: 98.11 min.

# Compute size factors



[1] "WARNING: Found size factors==0, setting these to 1."


# Fitting null and alternative model to the genes

Consumed time: 237.94 min.

# Differentially expression analysis based on model fits

Finished running ImpulseDE2.

TOTAL consumed time: 336.12 min.



In [12]:
fdr=0.05
result<-objectImpulseDE2$dfImpulseDE2Results
result<-result[result$padj<=fdr & ! is.na(result$padj),]
result<-result[order(result$padj),]
write.table(result,file=paste(pathSave,'DEacrossStages_padj',fdr,'.tsv',sep='') ,sep='\t',row.names = FALSE)

In [13]:
result

,Gene,p,padj,loglik_full,loglik_red,df_full,df_red,mean,converge_impulse,converge_const,allZero
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<lgl>
DDB_G0269574,DDB_G0269574,0.000000e+00,0.000000e+00,-3428.048,-4216.850,55,50,413.8111,0,0,FALSE
DDB_G0272076,DDB_G0272076,2.581236e-261,1.606819e-257,-4013.678,-4623.040,55,50,693.7308,0,0,FALSE
DDB_G0293844,DDB_G0293844,2.211433e-258,9.177449e-255,-3683.078,-4285.671,55,50,271.1776,0,0,FALSE
DDB_G0289749,DDB_G0289749,2.177071e-256,6.776134e-253,-3771.213,-4369.205,55,50,357.9575,0,0,FALSE
DDB_G0272104,DDB_G0272104,1.249398e-237,3.111001e-234,-4230.704,-4785.389,55,50,1168.0009,0,0,FALSE
DDB_G0278967,DDB_G0278967,7.256319e-236,1.505686e-232,-3986.953,-4537.566,55,50,512.5656,0,0,FALSE
DDB_G0283587,DDB_G0283587,9.130894e-235,1.623995e-231,-3691.103,-4239.176,55,50,307.6737,0,0,FALSE
DDB_G0289361,DDB_G0289361,4.674741e-230,7.275065e-227,-3908.599,-4445.799,55,50,406.4233,0,0,FALSE
DDB_G0275039,DDB_G0275039,1.871875e-225,2.589427e-222,-3916.328,-4442.900,55,50,775.9489,0,0,FALSE


In [17]:
head(result,100)

,Gene,p,padj,loglik_full,loglik_red,df_full,df_red,mean,converge_impulse,converge_const,allZero
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<lgl>
DDB_G0269574,DDB_G0269574,0.000000e+00,0.000000e+00,-3428.048,-4216.850,55,50,413.8111,0,0,FALSE
DDB_G0272076,DDB_G0272076,2.581236e-261,1.606819e-257,-4013.678,-4623.040,55,50,693.7308,0,0,FALSE
DDB_G0293844,DDB_G0293844,2.211433e-258,9.177449e-255,-3683.078,-4285.671,55,50,271.1776,0,0,FALSE
DDB_G0289749,DDB_G0289749,2.177071e-256,6.776134e-253,-3771.213,-4369.205,55,50,357.9575,0,0,FALSE
DDB_G0272104,DDB_G0272104,1.249398e-237,3.111001e-234,-4230.704,-4785.389,55,50,1168.0009,0,0,FALSE
DDB_G0278967,DDB_G0278967,7.256319e-236,1.505686e-232,-3986.953,-4537.566,55,50,512.5656,0,0,FALSE
DDB_G0283587,DDB_G0283587,9.130894e-235,1.623995e-231,-3691.103,-4239.176,55,50,307.6737,0,0,FALSE
DDB_G0289361,DDB_G0289361,4.674741e-230,7.275065e-227,-3908.599,-4445.799,55,50,406.4233,0,0,FALSE
DDB_G0275039,DDB_G0275039,1.871875e-225,2.589427e-222,-3916.328,-4442.900,55,50,775.9489,0,0,FALSE
